In [ ]:
%%configure -f
{"conf": {"spark.pyspark.python":"/home/hadoop/path/to/venv/bin/python3"}}

# <font color='darkblue'> Sparkify Full Dataset Models and Evaluation</font>

<div class="alert alert-block alert-info">

This notebook is dedicated to the following tasks:
    <li>load and clean data</li>
    <li>pre-process data and create new features</li>
    <li>stratified split features dataset into train and test sets</li>
    <li>fit several classifiers on train set, using default parameters</li>
    <li>tune two best classifiers using grid search and cross validation</li>
    <li>evaluate metrics and display all the results</li>
</div>

## <font color='blue'>Set Environment</font>

In [ ]:
# import PySpark libraries and packages

import pyspark
from pyspark import SparkContext
from pyspark import SparkConf

from pyspark.sql import SparkSession
from pyspark.sql.window import Window as W

from pyspark.sql.types import (
    StringType,
    IntegerType, 
    DateType, 
    TimestampType,
    )

from pyspark.sql.functions import (
    min as Fmin, max as Fmax, 
    sum as Fsum, round as Fround, 
    
    col, lit, 
    first, last, 
    desc, asc,
    avg, count, countDistinct, 
    when, isnull, isnan,
    from_unixtime, 
    datediff,
    )

# libraries and packages for modeling

from pyspark.ml import Pipeline

from pyspark.ml.feature import (
    StringIndexer, 
    OneHotEncoder, 
    VectorAssembler, 
    StandardScaler
)
from pyspark.ml.feature import (
    OneHotEncoder, 
    OneHotEncoderModel
)

from pyspark.ml.classification import (
    LogisticRegression,
    DecisionTreeClassifier,
    RandomForestClassifier,
    GBTClassifier,
    MultilayerPerceptronClassifier,
    LinearSVC,
    NaiveBayes
)

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
# build a Spark session using the SparkSession APIs

spark = (SparkSession
        .builder
        .appName("Sparkify")
        .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

In [ ]:
# check the available memory in the cluster
print('Available AWS-EMR Memory: {}'.format(spark.sparkContext
                                            .getConf().get('spark.driver.memory')))

In [ ]:
# import python libraries

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import sklearn metrics related packages

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc, roc_curve, roc_auc_score

In [ ]:
# import library for enhanced plotting

import seaborn as sns
sns.set_style("darkgrid")
colors = sns.color_palette('PuBuGn_r')

## <font color='blue'>Load, Preprocess, Process and Split Dataset Functions</font>

In [ ]:
def load_data(file_path):
    """
    Loads the raw dataset in Spark.
    
    INPUT:
            (str) - path for datafile
    OUTPUT:
            (PySpark dataframe) - dataframe of raw dat
    
    """
    
    print("Loading the dataset ...")
    df = spark.read.json(file_path)
    print("Dataset is loaded...")
    
    return df


def save_data(df, data_path):
    """
    Saves the PySpark dataframe to a file.
    
    INPUT:
            df (PySpark dataframe) - data to be saved
            data_path (str) - path for datafile
    OUTPUT:
            none
    
    """
    
    df.write.json(data_path)
    
    
def clean_data(df):
    """
    Performs basic cleaning operations on the raw data:
        - removes entries with missing userId
        - rescale timestamp columns to seconds
        - drop unnecesary columns
            - personal information columns
            - song information columns
            - web and browser information
            - timestamp columns in miliseconds

    INPUT:
        (PySpark dataframe) - dataframe of raw data
    OUTPUT:
        (PySpark dataframe) - dataframe of cleaned data
    """

    # print message to indicate the start of the process
    print("Cleaning the data ...")

    # print a count of rows before cleaning
    initial_records = df.count()
    print("Dataset has {} rows initially.".format(initial_records))

    # filter out all the records without userId
    df = df.where(df.userId != "")

    # rescale the timestamp to seconds (initially in miliseconds)
    df = df.withColumn("log_ts", df.ts/1000.0)
    df = df.withColumn("reg_ts", df.registration/1000.0)

    # drop several unnecessary columns
    cols_to_drop = ("firstName", "lastName", "location",
                    "artist", "song", "length",
                    "userAgent", "method", "status",
                    "ts", "registration"
                   )
    df = df.drop(*cols_to_drop)


    # print end of process message
    print("Finished cleaning the data ...")

    # print a count of rows after cleaning
    removed_rows = initial_records - df.count()

    print("Cleaned dataset has {} rows, {} rows were removed". \
        format(df.count(), initial_records - df.count()))

    return df

def preprocess_data(df):

    """
    Prepare the data for modeling via creating several features.

        - reg_date (date) - month-year of the registration

        - create windows grouped on userId and sessionId

         - firstevent_ts (timestamp) - first time an user is active
         - lastevent_ts (timestamp) - last time an user is active

         - init_days_interv (float) - days between registration and first activity
         - tenure_days_interv (float) - days between registration and last activity
         - active_days (float) - days the user has some activity on the platform
         - session_h (float) - session's duration in hours

     INPUT:
         df (PySpark dataframe) - cleaned dataframe
     OUTPUT:
         df (PySpark dataframe) - dataframe with the listed features added
    """

    # extract registration month and year from timestamp
    df = df.withColumn("reg_date", from_unixtime(col("reg_ts"), "MM-yyyy"))

    # create window: data grouped by userId, time ordered
    win_user = (W.partitionBy("userId")
            .orderBy("log_ts")
            .rangeBetween(W.unboundedPreceding, W.unboundedFollowing))

    # create window: data grouped by sessionId and userId, time ordered
    win_user_session = (W.partitionBy("sessionId", "userId")
                        .orderBy("log_ts")
                        .rangeBetween(W.unboundedPreceding, W.unboundedFollowing))

    # record the first time an user is active
    df = df.withColumn("firstevent_ts", first(col("log_ts")).over(win_user))
    # record the last time an user is active
    df = df.withColumn("lastevent_ts", last(col("log_ts")).over(win_user))

    # warmup time = registration time to first event in days
    df = df.withColumn("init_days_interv",
                       (col("firstevent_ts").cast("long")-col("reg_ts").cast("long"))/(24*3600))

    # tenure time = registration time to last event in days
    df = df.withColumn("tenure_days_interv",
                       (col("lastevent_ts").cast("long")-col("reg_ts").cast("long"))/(24*3600))

    # active time =  days between the first event and the last event in days
    df = df.withColumn("active_days",
                       (col("lastevent_ts").cast("long")-col("firstevent_ts").cast("long"))/(24*3600))

    # create column that records the individual session's duration in hours
    df = df.withColumn("session_h",
                    (last(df.log_ts).over(win_user_session) \
                     - first(df.log_ts).over(win_user_session))/3600)

    # drop columns
    df = df.drop("reg_ts", "log_ts")

    return df


def build_features(df):

    """
    Features engineered to be used in modelling.

        - nr_songs (int) - total number of songs user listened to
        - nr_playlist (int) - number of songs added to the playlist

        - nr_friends (int) - number of friends added through "Add Friend"

        - nr_likes (int) - total number of "Thumbs Up" of the user
        - nr_dislikes (int) - total number of "Thumbs Down" of the user

        - nr_downgrades (int) - total number of visits to "Downgrade" page by the user
        - nr_upgrades (int) - total number of visits to "Upgrade" page by the user

        - nr_home (int) - total number of visits to "Home" page by the user
        - nr_settings (int) - total number of visits to "Settings" page by the user

        - nr_error (int) - total number of errors encountered by the user

        - nr_ads (int) - total number of ads the user got
        - nr_sessions (int) - number of sessions of the user
        - n_acts (int) - total number of actions taken by the user

        - avg_sess_h (float) - average session length in hours
        - acts_per_session (float) - average number of actions per session for the user
        - songs_per_session (float) - average numer of songs listened per session by the user
        - ads_per_session (float) - average number of ads per session, received by user

        - init_days_interv (int) - time interval in days from registration to the first action of the user
        - tenure_days_interv (int) - time interval in days from registration to the last action of the user
        - active_days (int) - number of days the user was active on the platform

        - gender (binary) - 1 for F (female), 0 for M (male)
        - level (binary) - 1 for paid, 0 for free

        - churn (binary) - 1 for "Cancellation Confirmation" page visit, 0 otherwise

    INPUT:
        df (PySpark dataframe) - preprocessed dataframe
    OUTPUT:
        df_feats (PySpark dataframe) - dataframe that contains engineered features
    """

    df_feats = df.groupBy("userId") \
        .agg(

            # count user's individual actions using all page visits

            count(when(col("page") == "NextSong", True)).alias("nr_songs"),
            count(when(col("page") == "Add to Playlist", True)).alias("nr_playlist"),

            count(when(col("page") == "Add Friend", True)).alias("nr_friends"),

            count(when(col("page") == "Thumbs Up", True)).alias("nr_likes"),
            count(when(col("page") == "Thumbs Down", True)).alias("nr_dislikes"),

            count(when(col("page") == "Downgrade", True)).alias("nr_downgrades"),
            count(when(col("page") == "Upgrade", True)).alias("nr_upgrades"),

            count(when(col("page") == "Home", True)).alias("nr_home"),
            count(when(col("page") == "Settings", True)).alias("nr_settings"),

            count(when(col("page") == "Error", True)).alias("nr_error"),

            count(when(col("page") == "Roll Advert", True)).alias("nr_ads"),

            # compute the number of sessions a user is in
            countDistinct("sessionId").alias("nr_sessions"),

            # find the total number of actions a user took
            countDistinct("itemInSession").alias("n_acts"),

            # compute the average session length in hours
            avg(col("session_h")).alias("avg_sess_h"),

            # compute the average number of page actions per sesssion - i.e. items in session
            (countDistinct("itemInSession")/countDistinct("sessionId")).alias("acts_per_session"),

            # compute the average number of songs per session
            (count(when(col("page") == "NextSong",
                        True))/countDistinct("sessionId")).alias("songs_per_session"),

            # compute the average number of ads per session
             (count(when(col("page") == "Roll Advert",
                         True))/countDistinct("sessionId")).alias("ads_per_session"),

            # days between registration and first activity
            first(col("init_days_interv")).alias("init_days_interv"),
            # the tenure time on the platform: from registration to last event in days
            first(col("tenure_days_interv")).alias("tenure_days_interv"),
            # number of days user visited the platform, is active on the platform
            first(col("active_days")).alias("active_days"),

            # encode the gender 1 for F and 0 for M
            first(when(col("gender") == "F", 1).otherwise(0)).alias("gender"),

            # encode the level (paid/free) according to the last record
            last(when(col("level") == "paid", 1).otherwise(0)).alias("level"),

            # flag those users that downgraded
            #last(when(col("page") == "Downgrade", 1).otherwise(0)).alias("downgrade"),

            # create the churn column that records if the user cancelled
            last(when(col("page") == "Cancellation Confirmation", 1).otherwise(0)).alias("churn"),
            )

    # columns to drop
    drop_cols = ("userId", "gender", "avg_sess_h",
                 "nr_playlist", "nr_home")
    # drop the columns
    #df_feats = df_feats.drop("userId")
    df_feats = df_feats.drop(*drop_cols)

    # drop the null values
    df_feats=df_feats.na.drop()

    return df_feats


SPLIT_VALS = [.7, .3]

# split the data into train and test sets

def split_data (df):

    """
    Split the dataset into training set and test set.
    Use a stratified sampling method.

    INPUT:
        df (PySpark dataframe) - dataframe
    OUTPUT:
        train_set, test_set (PySpark dataframes) - percentage split based on the provided values
    """

    # split dataframes between 0s and 1s
    zeros = df.filter(df["churn"]==0)
    ones = df.filter(df["churn"]==1)

    # split dataframes into training and testing
    train0, test0 = zeros.randomSplit(SPLIT_VALS, seed=1234)
    train1, test1 = ones.randomSplit(SPLIT_VALS, seed=1234)

    # stack datasets back together
    train_set = train0.union(train1)
    test_set = test0.union(test1)

    return train_set, test_set

## <font color='blue'>Load and Clean Data, Create Feature Dataset and Split Data</font>

In [ ]:
##TEST
# path for the train set file
path_mini = "data/mini_sparkify_event_data.json"

# upload the train data
df = load_data(path_mini)

In [ ]:
# create a path variable for the full dataset file
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"

In [ ]:
# start timer
start_time = time.time()

# load the dataset 
df = load_data(event_data)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
load_time = (end_time - start_time)/60

# print the loading time
print("")
print("Load time.........%6.3f min" % load_time)


In [ ]:
# set memory protocol for spark
df_cached = df.cache()

In [ ]:
# start timer
start_time = time.time()

# clean the raw dataset
df_clean = clean_data(df_cached)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
clean_time = (end_time - start_time)/60

# print the training time
print("")
print("Cleaning time.........%6.3f min" % clean_time)


In [ ]:
# start timer
start_time = time.time()

# preprocess the dataset
df_proc = preprocess_data(df_clean)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
preproc_time = (end_time - start_time)/60

# print the training time
print("")
print("Preprocessing time.........%6.3f min" % preproc_time)


In [ ]:
# start timer
start_time = time.time()

# create features dataset
df_feats = build_features(df_proc)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
feat_time = (end_time - start_time)/60

# print the training time
print("")
print("Features engineering time.........%6.3f min" % feat_time)

In [ ]:
# split the features dataset
df_train, df_test = split_data(df_feats)

### <font color='blue'>Toggle the memory</font>

In [ ]:
# add the train set to the memory
df_cached.unpersist()

In [ ]:
# add the train set to the memory
train_cached = df_train.cache()

In [ ]:
# add the test set to the memory
test_cached = df_test.cache()

## <font color='blue'>Baseline Model</font>

In [ ]:
# count the number of churn users in each set
fn_train = train_cached.where(train_cached.churn==1).count()
fn_test = test_cached.where(test_cached.churn==1).count()

# count the number of not churn users in each set
tn_train = train_cached.where(train_cached.churn==0).count()
tn_test = test_cached.where(test_cached.churn==0).count()

In [ ]:
# accuracy of ZeroR model on train set 
accuracy_train = tn_train/(fn_train+tn_train)
# print the result
print("With fp = tp = 0, fn = {} and tn = {}, the accuracy of the ZeroR model on the train set is {}%."
      .format(fn_train, tn_train, round(accuracy_train,4)*100))

# accuracy of ZeroR model on test set
accuracy_test = tn_test/(fn_test+tn_test)
# print the result
print("With fp = tp = 0, fn = {} and tn = {}, the accuracy of the ZeroR model on the test set is {}%."
     .format(fn_test, tn_test, round(accuracy_test,4)*100))

### <font color='blue'>Build Models Evaluators</font>

In [ ]:
# function to compute relevant metrics for binary classification
def conf_metrics(dataset):
    
    """
        Calculates the metrics associated to the confusion matrix.

        INPUT:
            dataset (pyspark.sql.DataFrame) - a dataset that contains
                                labels and predictions
        OUTPUT:
            accuracy (float) - metric
            precision (float) - metric
            recall (float) - metric
            F1 (float) - metric
    """
   

    # calculate the elements of the confusion matrix
    tn = dataset.where((dataset[labelCol]==0) & (dataset[predCol]==0)).count()
    tp = dataset.where((dataset[labelCol]==1) & (dataset[predCol]==1)).count()                   
    fn = dataset.where((dataset[labelCol]==1) & (dataset[predCol]==0)).count()                   
    fp = dataset.where((dataset[labelCol]==0) & (dataset[predCol]==1)).count()
    
    # calculate accuracy, precision, recall, and F1-score
    accuracy = (tn + tp) / (tn + tp + fn + fp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 =  2 * (precision*recall) / (precision + recall)
    
    return accuracy, precision, recall, f1

In [ ]:
# function to display the metrics of interest
def display_metrics(dataset, roc_cl, pr_cl):
    
    """
    Prints evaluation metrics for the model. 
    
    INPUT:
         dataset (pyspark.sql.DataFrame) - a dataset that contains
                                labels and predictions
    
    """
    
    accuracy = conf_metrics(dataset)[0]
    precision = conf_metrics(dataset)[1]
    recall = conf_metrics(dataset)[2]
    f1 = conf_metrics(dataset)[3]
    
    print("")
    print("Confusion Matrix")
    dataset.groupBy(dataset[labelCol], dataset[predCol]).count().show()
    print("")
    print("accuracy...............%6.3f" % accuracy)
    print("precision..............%6.3f" % precision)
    print("recall.................%6.3f" % recall)
    print("F1.....................%6.3f" % f1)
    print("auc_roc................%6.3f" % roc_cl)
    print("auc_pr.................%6.3f" % pr_cl)


In [ ]:
# function to plot heatmap for a confusion matrix

def conf_mat_heatmap(predictions, model_name):
    
    """
    Uses confusion matrix utility in sklearn to print a heatmap.
    
    INPUT:
        predictions (PySpark dataframe) - contains probability predictions, label column
        model_name (str) - abreviated classifier name
        
    OUTPUT:
        none - confusion matrix heatmap is displayed
        
    """
    
    # transform predictions PySpark dataframe into Pandas dataframe
    pred_pandas = predictions.select(predictions[labelCol], predictions[predCol]).toPandas()
    
    # to avoid stacking plots
    plt.clf()
    
    # generate the confusion matrix
    ConfusionMatrixDisplay.from_predictions(pred_pandas[labelCol], pred_pandas[predCol], 
                                            cmap="Blues", values_format='d', ax=None)
    
    
    # figure title
    plt.title("Confusion Matrix:" + model_name)

    # display the plot
    %matplot plt

In [ ]:
# function to plot the roc and pr curves side by side
def plot_roc_pr_curves(predictions, model_name):
    
    """
    Calculates ROC-AUC and PR-AUC scores and plots the ROC and PR curves.
    
    INPUT:
        predictions (PySpark dataframe) - contains probability predictions, label column
        model_name (str) - classifier name
        
    OUTPUT:
        none - two plots are displayed
        
    """
    
    # transform predictions PySpark dataframe into Pandas dataframe
    pred_pandas = predictions.select(predictions.label, predictions.probability).toPandas()
    
    # calculate roc_auc score
    roc_auc = roc_auc_score(pred_pandas.label, pred_pandas.probability.str[1])
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(pred_pandas.label))]
    # calculate roc curves
    fpr, tpr, _ = roc_curve(pred_pandas.label, pred_pandas.probability.str[1])
    ns_fpr, ns_tpr, _ = roc_curve(pred_pandas.label, ns_probs)
    
    # calculate precision, recall for each threshold
    precision, recall, _ = precision_recall_curve(pred_pandas.label, pred_pandas.probability.str[1])
    # calculate pr auc score
    pr_auc = auc(recall, precision)
    
    # to avoid stacking plots
    plt.clf()
    
    # create figure which contains two subplots
    plt.figure(figsize=[12,6])
    
    plt.subplot(121)
    
    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(fpr, tpr, marker='.', color="firebrick", label='ROC AUC = %.3f' % (roc_auc))
    
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # figure title
    plt.title("ROC Curve:" + model_name)
    
    plt.subplot(122)
    
    # plot the precision-recall curves
    
    ns_line = len(pred_pandas[pred_pandas.label==1]) / len(pred_pandas.label)
    plt.plot([0, 1], [ns_line, ns_line], linestyle='--', label='No Skill')
    plt.plot(recall, precision, marker='.', color="firebrick", label='PR AUC = %.3f' % (pr_auc))
    
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend()
    # figure title
    plt.title("Precision-Recall Curve:" + model_name)

    # display the plot
    %matplot plt

### <font color='blue'>Build Pipelines</font>

In [ ]:
# split the features and the label
CAT_FEATURES = ["level"]
CONT_FEATURES = ["nr_songs", "nr_likes", "nr_dislikes", "nr_friends", "nr_downgrades",
                "nr_upgrades", "nr_error", "nr_settings", "nr_ads", "nr_sessions",
                "n_acts", "acts_per_session", "songs_per_session", "ads_per_session",
                "init_days_interv", "tenure_days_interv", "active_days"]
CHURN_LABEL = "churn"

# create labels and features
predCol="prediction"
labelCol="label"
featuresCol = "features"

In [ ]:
def build_full_pipeline(classifier):
    """
    Combines all the stages of the processing and modeling.
    """
    # stages in the pipeline
    stages = [] 
    
    # encode the labels
    label_indexer =  StringIndexer(inputCol=CHURN_LABEL, outputCol="label")
    stages += [label_indexer]
    
    # encode the binary features
    bin_assembler = VectorAssembler(inputCols=CAT_FEATURES, outputCol="bin_features")
    stages += [bin_assembler]
    
    # encode the continuous features
    cont_assembler = VectorAssembler(inputCols = CONT_FEATURES, outputCol="cont_features")
    stages += [cont_assembler]
    # normalize the continuous features
    cont_scaler = StandardScaler(inputCol="cont_features", outputCol="cont_scaler", 
                                 withStd=True , withMean=True)
    stages += [cont_scaler]
    
    # pass all to the vector assembler to create a single sparse vector
    all_assembler = VectorAssembler(inputCols=["bin_features", "cont_scaler"],  
                            outputCol="features")
    stages += [all_assembler]
    
    # add the model to the pipeline
    stages += [classifier]
    
    # create a pipeline
    pipeline = Pipeline(stages=stages)
    
    return pipeline

In [ ]:
# implement K-fold cross validation and grid search 

def grid_search_model(pipeline, param):
    """
    Creates a cross validation object and performs grid search
    over a set of parameters.
    
    INPUT:
        param = grid of parameters
        pipeline = model pipeline 
    
    OUTPUT:
        cv = cross validation object
    """
    evaluator = BinaryClassificationEvaluator()
    cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=param,
                    evaluator=evaluator,
                    numFolds=5,
                    parallelism=2)
    return cv

## <font color='blue'>Evaluate PySpark Classifiers</font>

<div class="alert alert-block alert-info">

Fit the binary classifiers from PySpark on the train set with cross validation and default parameters to select the best ones for the next stage.
   
</div>

### <font color='blue'>Logistic Regression Classifier</font>

In [ ]:
print("")
print(f"Training LOGISTIC REGRESSION CLASSIFIER")
print("")

# instantiate the classifier
lr_classifier = LogisticRegression(labelCol = labelCol,
                                       featuresCol = featuresCol)

# build specific pipeline
lr_pipeline = build_full_pipeline(lr_classifier)

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# start timer
start_time = time.time()

# train the model
model_lr = lr_pipeline.fit(train_cached)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_lr = model_lr.transform(train_cached)

# calculate auc metrics
roc_lr = evaluator.evaluate(predictions_lr, {evaluator.metricName: "areaUnderROC"})
pr_lr = evaluator.evaluate(predictions_lr, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_lr, prec_lr, rec_lr, f1_lr = conf_metrics(predictions_lr)

# print all evaluation metrics
print("")
print("LR: Train set evaluation metrics")
display_metrics(predictions_lr, roc_lr, pr_lr)
print("")

# plot heatmap for confusion matrix
conf_mat_heatmap(predictions_lr, "LR")

# plot the ROC and PR curves
plot_roc_pr_curves(predictions_lr, "LR")

### <font color='blue'>Decision Trees Classifier</font>

In [ ]:
print("")
print(f"Training DECISION TREES CLASSIFIER")
print("")

predCol="prediction"
labelCol="label"

# build specific pipeline
dt_classifier = DecisionTreeClassifier(labelCol = labelCol,
                                           featuresCol = featuresCol,
                                           seed=1234)
dt_pipeline = build_full_pipeline(dt_classifier)


# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# start timer
start_time = time.time()

# train the model
model_dt = dt_pipeline.fit(train_cached)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_dt = model_dt.transform(train_cached)

# calculate auc metrics
roc_dt = evaluator.evaluate(predictions_dt, {evaluator.metricName: "areaUnderROC"})
pr_dt = evaluator.evaluate(predictions_dt, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_dt, prec_dt, rec_dt, f1_dt = conf_metrics(predictions_dt)

# print all evaluation metrics
print("")
print("DT: Train set evaluation metrics")
display_metrics(predictions_dt, roc_dt, pr_dt)
print("")

# plot heatmap for confusion matrix
conf_mat_heatmap(predictions_dt, "DT")

# plot the ROC and PR curves
plot_roc_pr_curves(predictions_dt, "DT")

### <font color='blue'>Random Forest Classifier</font>

In [ ]:
print("")
print(f"Training RANDOM FOREST CLASSIFIER")
print("")

predCol="prediction"
labelCol="label"

# instantiate the classifier
rf_classifier = RandomForestClassifier(labelCol = labelCol,
                                       featuresCol = featuresCol, 
                                       seed=1234)
# build the specific pipeline
rf_pipeline = build_full_pipeline(rf_classifier)

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# start timer
start_time = time.time()

# train the model
model_rf = rf_pipeline.fit(train_cached)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_rf = model_rf.transform(train_cached)

# calculate auc metrics
roc_rf = evaluator.evaluate(predictions_rf, {evaluator.metricName: "areaUnderROC"})
pr_rf = evaluator.evaluate(predictions_rf, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_rf, prec_rf, rec_rf, f1_rf = conf_metrics(predictions_rf)

# print all evaluation metrics
print("")
print("RF: Train set evaluation metrics")
display_metrics(predictions_rf, roc_rf, pr_rf)
print("")

# plot heatmap for confusion matrix
conf_mat_heatmap(predictions_rf, "RF")

# plot the ROC and PR curves
plot_roc_pr_curves(predictions_rf, "RF")

### <font color='blue'>Gradient Boosted Trees</font>

In [ ]:
print("")
print(f"Training GRADIENT BOOSTED TREES")
print("")

# instantiate the classifier
gbt_classifier = GBTClassifier(labelCol = labelCol,
                                featuresCol = featuresCol,
                                seed=1234)
# build specific pipeline
gbt_pipeline = build_full_pipeline(gbt_classifier)

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# start timer
start_time = time.time()

# train the model
model_gbt = gbt_pipeline.fit(train_cached)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_gbt = model_gbt.transform(train_cached)

# calculate auc metrics
roc_gbt = evaluator.evaluate(predictions_gbt, {evaluator.metricName: "areaUnderROC"})
pr_gbt = evaluator.evaluate(predictions_gbt, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_gbt, prec_gbt, rec_gbt, f1_gbt = conf_metrics(predictions_gbt)

# print all evaluation metrics
print("")
print("GBT: Train set evaluation metrics")
display_metrics(predictions_gbt, roc_gbt, pr_gbt)
print("")

# plot heatmap for confusion matrix
conf_mat_heatmap(predictions_gbt, "GBT")

# plot the ROC and PR curves
plot_roc_pr_curves(predictions_gbt, "GBT")

### <font color='blue'>Multilayer Perceptron Classifier</font>

In [ ]:
print("")
print(f"Training MULTILAYER PERCEPTRON CLASSIFIER")
print("")

# specify layers: 18 (features), two intermediate (8, 4), output 2 (classes)
layers=[18, 8, 4, 2]
# create the trainer and set its parameters
mlpc_classifier = MultilayerPerceptronClassifier(labelCol = labelCol,
                                                featuresCol = featuresCol,
                                                layers=layers,
                                                seed=1234)
# build specific pipeline
mlpc_pipeline = build_full_pipeline(mlpc_classifier)

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# start timer
start_time = time.time()

# train the model
model_mlpc = mlpc_pipeline.fit(train_cached)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_mlpc = model_mlpc.transform(train_cached)

# calculate auc metrics
roc_mlpc = evaluator.evaluate(predictions_mlpc, {evaluator.metricName: "areaUnderROC"})
pr_mlpc = evaluator.evaluate(predictions_mlpc, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_mlpc, prec_mlpc, rec_mlpc, f1_mlpc = conf_metrics(predictions_mlpc)

# print all evaluation metrics
print("")
print("MLPC: Train set evaluation metrics")
display_metrics(predictions_mlpc, roc_mlpc, pr_mlpc)
print("")

# plot heatmap for confusion matrix
conf_mat_heatmap(predictions_mlpc, "MLPC")

# plot the ROC and PR curves
plot_roc_pr_curves(predictions_mlpc, "MLPC")

### <font color='blue'>Linear Support Vector Machine</font>

In [ ]:
print("")
print(f"Training LINEAR SUPPORT VECTOR MACHINE")
print("")

# instantiate the classifier
lsvc_classifier = LinearSVC(labelCol = labelCol,
                            featuresCol = featuresCol)
# build specific pipeline
lsvc_pipeline = build_full_pipeline(lsvc_classifier)

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# start timer
start_time = time.time()

# train the model
model_lsvc = lsvc_pipeline.fit(train_cached)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_lsvc = model_lsvc.transform(train_cached)

# calculate auc metrics
roc_lsvc = evaluator.evaluate(predictions_lsvc, {evaluator.metricName: "areaUnderROC"})
pr_lsvc = evaluator.evaluate(predictions_lsvc, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_lsvc, prec_lsvc, rec_lsvc, f1_lsvc = conf_metrics(predictions_lsvc)

# print all evaluation metrics
print("")
print("LSVC: Train set evaluation metrics")
display_metrics(predictions_lsvc, roc_lsvc, pr_lsvc)
print("")

# plot heatmap for confusion matrix
conf_mat_heatmap(predictions_lsvc, "LSVC")

### <font color='blue'>Choose the Best Classifiers</font>

In [ ]:
# create Pandas dataframe with metrics
dict_metrics = {"LinReg": [acc_lr, prec_lr, rec_lr, f1_lr, roc_lr, pr_lr],
                "DecTrees": [acc_dt, prec_dt, rec_dt, f1_dt, roc_dt, pr_dt], 
                "RandForest": [acc_rf, prec_rf, rec_rf, f1_rf, roc_rf, pr_rf],
                "GradBoost": [acc_gbt, prec_gbt, rec_gbt, f1_gbt, roc_gbt, pr_gbt],
                "MultiLPerceptron": [acc_mlpc, prec_mlpc, rec_mlpc, f1_mlpc, roc_mlpc, pr_mlpc],
                "LinearSVM": [acc_lsvc, prec_lsvc, rec_lsvc, f1_lsvc, roc_lsvc, pr_lsvc],
                "list_metrics" : ["accuracy", "precision", "recall", "f1_score", "auc_roc", "auc_pr"]
               }
df_mets = pd.DataFrame.from_dict(dict_metrics).set_index("list_metrics")
df_mets.round(3)

In [ ]:
# to avoid stacking the plots
plt.clf()

# adjust figure size and font size
sns.set(rc = {"figure.figsize":(12,4)})

sns.set(font_scale=1)
ax = df_mets.plot.bar(y=["LinReg", "DecTrees", 
                         "RandForest", "GradBoost", "MultiLPerceptron", "LinearSVM"], rot=0)
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")

# create title and labels
plt.title("Confusion Matrix Metrics For Sparkify Small Train Dataset")
ax.set_xlabel("");
ax.set_ylabel("");

# display the plot
%matplot plt

## <font color='blue'>Tune Hyperparameters</font>

### <font color='blue'>Gradient Boosted Trees</font>

In [ ]:
# model_name = "Gradient Boosted Trees"
def gbt_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    # create a list of parameters for Gradient Boosted Trees
    param_gbt = ParamGridBuilder()
    param_gbt = param_gbt.addGrid(model.maxDepth, [10, 20, 30])
    param_gbt = param_gbt.addGrid(model.maxIter, [10, 20, 40]) 
    param_gbt = param_gbt.addGrid(model.stepSize, [.05, .1])
    param_gbt = param_gbt.build()
    
    print(f"Models trained: {len(param_gbt)}")
    
    return grid_search_model(pipeline, param_gbt)

In [ ]:
print("")
print(f"Training GRADIENT BOOSTED TREES")
print("")

# instantiate the classifier
gbt_classifier = GBTClassifier(labelCol = labelCol,
                                featuresCol = featuresCol,
                                seed=1234)
# build specific pipeline
gbt_pipeline = build_full_pipeline(gbt_classifier)

# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# build the grid search pipeline
gbt = gbt_grid_search(gbt_pipeline)

# start timer
start_time = time.time()

# train the model
model_gbt = gbt.fit(train_cached)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_gbt = model_gbt.bestModel.transform(train_cached)

# calculate auc metrics
roc_gbt = evaluator.evaluate(predictions_gbt, {evaluator.metricName: "areaUnderROC"})
pr_gbt = evaluator.evaluate(predictions_gbt, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_gbt, prec_gbt, rec_gbt, f1_gbt = conf_metrics(predictions_gbt)

# print all evaluation metrics
print("")
print("GBT: Evaluation metrics on the test set")
display_metrics(predictions_gbt, roc_gbt, pr_gbt)
print("")

# print the best parameters from the grid
best_model_gbt = model_gbt.bestModel.stages[-1]
param_gbt1 = best_model_gbt.getMaxDepth()
param_gbt2 = best_model_gbt.getMaxIter()
param_gbt3 = best_model_gbt.getStepSize()

print("The best hyperparameter values from the grid:")
print("maxDepth:..........", param_gbt1)
print("maxIter:...........", param_gbt2)
print("stepSize:..........", param_gbt3)

# plot heatmap for confusion matrix
conf_mat_heatmap(predictions_gbt, "GBT")

# plot the ROC and PR curves
plot_roc_pr_curves(predictions_gbt, "GBT")

### <font color='blue'>Multilayer Perceptron Classifier</font>

In [ ]:
# model_name = "Multilayer Perceptron Classifier"
def mlpc_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    # create a list of parameters for Multilayer Perceptron Classifier
    param_mlpc = ParamGridBuilder()
    param_mlpc = param_mlpc.addGrid(model.stepSize, [.05, .1])
    param_mlpc = param_mlpc.addGrid(model.maxIter, [100, 200]) 
    param_mlpc = param_mlpc.addGrid(model.blockSize, [64, 128, 256]) 
    param_mlpc = param_mlpc.build()
    
    print(f"Models trained: {len(param_mlpc)}")
    
    return grid_search_model(pipeline, param_mlpc)

In [ ]:
print("")
print(f"Training MULTILAYER PERCEPTRON CLASSIFIER")
print("")

# specify layers: 18 (features), two intermediate (8, 4), output 2 (classes)
layers=[18, 8, 4, 2]
# create the trainer and set its parameters
mlpc_classifier = MultilayerPerceptronClassifier(labelCol = labelCol,
                                                featuresCol = featuresCol,
                                                layers=layers,
                                                seed=1234)
# build specific pipeline
mlpc_pipeline = build_full_pipeline(mlpc_classifier)


# choose an evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol(labelCol)

# build the grid search pipeline
mlpc = mlpc_grid_search(mlpc_pipeline)

# start timer
start_time = time.time()

# train the model
model_mlpc = mlpc.fit(train_cached)

# stop timer
end_time = time.time()

# evaluate the trainining time in minutes 
train_time = (end_time - start_time)/60

# print the training time
print("")
print("Training time.........%6.3f min" % train_time)

# create the predictions dataset
predictions_mlpc = model_mlpc.bestModel.transform(train_cached)

# calculate auc metrics
roc_mlpc = evaluator.evaluate(predictions_mlpc, {evaluator.metricName: "areaUnderROC"})
pr_mlpc = evaluator.evaluate(predictions_mlpc, {evaluator.metricName: "areaUnderPR"})

# record the confusion matrix metrics
acc_mlpc, prec_mlpc, rec_mlpc, f1_mlpc = conf_metrics(predictions_mlpc)

# print all evaluation metrics
print("")
display_metrics(predictions_mlpc, roc_mlpc, pr_mlpc)
print("")

# print the best parameters from the grid
best_model_mlpc = model_mlpc.bestModel.stages[-1]
param_mlpc1 = best_model_mlpc.getStepSize()
param_mlpc2 = best_model_mlpc.getMaxIter()
param_mlpc3 = best_model_mlpc.getBlockSize()
print("The best hyperparameter values from the grid:")
print("stepSize:..........", param_mlpc1)
print("maxIter:...........", param_mlpc2)
print("blockSize:.........", param_mlpc3)

# plot heatmap for confusion matrix
conf_mat_heatmap(predictions_lsvc, "LSVC")

# plot the ROC and PR curves
plot_roc_pr_curves(predictions_mlpc, "MLPC")

### <font color='blue'>Compare Tuned Classifiers</font>

In [ ]:
def plot_roc_pr_curves(predictions_model1, predictions_model2, model1, model2):
    
    """
    Plots the ROC and PR curves for two models on the same graphs.
    
    INPUT:
        predictions_model1 (PySpark dataframe) - contains probability predictions for the first model
        predictions_model2 (PySpark dataframe) - contains probability predictions for the second model
        model1, model2 (str) - abreviated model names
        
    OUTPUT:
        none - two plots are displayed
        
    """
    
    # transform predictions PySpark dataframe into Pandas dataframe
    pred1_pandas = predictions_model1.select(predictions_model1.label, 
                                             predictions_model1.probability).toPandas()
    pred2_pandas = predictions_model2.select(predictions_model2.label, 
                                             predictions_model2.probability).toPandas()
    
    # calculate roc_auc scores
    roc_auc1 = roc_auc_score(pred1_pandas.label, pred1_pandas.probability.str[1])
    # calculate roc_auc scores
    roc_auc2 = roc_auc_score(pred2_pandas.label, pred2_pandas.probability.str[1])
    
    # calculate roc curves for model 1
    fpr1, tpr1, _ = roc_curve(pred1_pandas.label, pred1_pandas.probability.str[1])
    # calculate roc curves for model 2
    fpr2, tpr2, _ = roc_curve(pred2_pandas.label, pred2_pandas.probability.str[1])
   
    # calculate precision, recall for each threshold for the first model
    precision1, recall1, _ = precision_recall_curve(pred1_pandas.label, pred1_pandas.probability.str[1])
    # calculate pr auc score for the first model
    pr_auc1 = auc(recall1, precision1)
    
    # calculate precision, recall for each threshold for the second model
    precision2, recall2, _ = precision_recall_curve(pred2_pandas.label, pred2_pandas.probability.str[1])
    # calculate pr auc score for the second model
    pr_auc2 = auc(recall2, precision2)
    
    plt.clf()
    
    # create figure which contains two subplots
    plt.figure(figsize=[12,6])
    
    plt.subplot(121)
    
    # plot the roc curve for the model1
    plt.plot(ns_fpr1, ns_tpr1, linestyle='--', label='No Skill')
    plt.plot(fpr1, tpr1, marker='.', color='firebrick', label=model1 + 'ROC AUC = %.3f' % (roc_auc1))
    plt.plot(fpr2, tpr2, marker='.', color='green', label=model2 + 'ROC AUC = %.3f' % (roc_auc2))
    
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # figure title
    plt.title("ROC Curves")
    
    plt.subplot(122)
    
    # plot the precision-recall curves
    
    #ns_line = len(pred_pandas[pred_pandas.label==1]) / len(pred_pandas.label)
    #plt.plot([0, 1], [ns_line, ns_line], linestyle='--', label='No Skill')
    plt.plot(recall1, precision1, marker='.', color="firebrick", label=model1+ 'PR AUC = %.3f' % (pr_auc1))
    plt.plot(recall2, precision2, marker='.', color="green", label=model2+'PR AUC = %.3f' % (pr_auc2))
    
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend()
    # figure title
    plt.title("Precision-Recall Curves")

    # display the plot
    %matplot plt

In [ ]:
# plot the two pairs of curves for comparison
plot_roc_pr_curves(predictions_mlpc, predictions_gbt, "MLPC", "GBT")

In [ ]:
# create Pandas dataframe with evaluation metrics for the best hyperparameters selection
dict_metrics = {"GradBoost": [acc_gbt, prec_gbt, rec_gbt, f1_gbt, roc_gbt, pr_gbt],
                "MultiLPerceptron": [acc_mlpc, prec_mlpc, rec_mlpc, f1_mlpc, roc_mlpc, pr_mlpc],
                "list_metrics" : ["accuracy", "precision", "recall", "f1_score", "auc_roc", "auc_pr"]
               }
df_mets = pd.DataFrame.from_dict(dict_metrics).set_index("list_metrics")
df_mets.round(3)

In [ ]:
# barplot of the evaluation metrics for the two models, for comparison

plt.clf()

# adjust figure size and font size
sns.set(rc = {"figure.figsize":(12,2)})

sns.set(font_scale=1)
ax = df_mets.plot.bar(y=["GradBoost", "MultiLayerPerceptron"], rot=0)
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")_

# create title and labels
plt.title("Evaluation Metrics For Sparkify Full Test Dataset")
ax.set_xlabel("");
ax.set_ylabel("");

# display the plot
%matplot plt